# Bayesian Linear Regression - Solution Notebook

In this session, you'll implement Bayesian linear regression with basis functions.

In [ ]:
import functools
import warnings

import matplotlib
import matplotlib.font_manager
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg
import scipy.stats
from matplotlib import rc
from mpl_toolkits.mplot3d import Axes3D

preamble = r"""\renewcommand{\familydefault}{\sfdefault}
            \usepackage{sansmath} \sansmath  \usepackage{amsmath}"""
rc("font", **{"family": "sans-serif", "sans-serif": "DejaVu Sans"})
rc("text", **{"usetex": False, "latex.preamble": preamble})
rc("figure", **{"dpi": 200})
rc(
    "axes",
    **{"spines.right": False, "spines.top": False, "xmargin": 0, "ymargin": 0.05}
)

## 1. A Regression Dataset
Create a simple 1D regression dataset using the `make_regression(...)` function and plot it.
For the moment, keep the noise variance $\sigma_\mathrm{n}$ small.
NB. For better reproducibility, please remember to fix the Numpy's random seed. 
For Jupyter notebooks, this needs to be done at the beginning of each cell.

In [ ]:
true_function = lambda x: 0.5 * ((x - 1) ** 2) - 3


def make_regression(n, sn2=0.1):
    x = np.random.uniform(-3, 3, n)
    y = true_function(x) + np.sqrt(sn2) * np.random.randn(*x.shape)
    return x, y

In [ ]:
np.random.seed(1)
sn2 = 1.5
x, y = make_regression(30, sn2=sn2)
xp = np.linspace(-4, 4)

fig, ax = plt.subplots(figsize=[5, 3], dpi=250)
ax.plot(xp, true_function(xp), "--k", zorder=0, label="True function")
ax.scatter(x, y, edgecolor="black", linewidth=1, facecolor="xkcd:orange")
ax.set_title("A regression dataset")
ax.legend()
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.margins(0.05)
plt.show()

## 2. A review on the Gaussian likelihood
Let's start from the basics. Remember what a likelihood is.
The **likelihood** measures the goodness of fit of a statistical model to data for given values of 
the unknown model parameters.
It's computed from the joint probability distribution, but viewed and used as **function** 
of the parameters only, thus treating the random variables as fixed at the observed values.

A Gaussian likelihood is defined as: 

\begin{equation}
p(\mathbf{y}|\mathbf{w}, \mathbf{X}, \sigma_\mathrm{n}) = \prod_{i=1}^N p(y_i|\mathbf{w}, {\mathbf{x}}_i, \sigma_\mathrm{n}) = \prod_{i=1}^N \mathcal{N}(y_i|\tilde y_i, \sigma_\mathrm{n})
\end{equation}

where, for linear regression, $\tilde y_i = \mathbf{w}^\top {\mathbf{x}}_i$.
For numerical stability, instead of using the likelihood, we will use the **log-likelihood**.

\begin{equation}
\log p(\mathbf{y}|\mathbf{w}, \mathbf{X}, \sigma_\mathrm{n}) =  \sum_{i=1}^N  \log\mathcal{N}(y_i|\tilde y_i, \sigma_\mathrm{n})
\end{equation}

**Exercise:**
Write a function to compute the log-density of a normal distribution at position $x$, given $\mu$ and $\sigma^2$.

In [ ]:
def lognormal(x, mu, var):
    # @@ COMPLETE @@ #
    return

**Exercise:**
For the moment, assume that for the sample $i^{\mathrm{th}}$, you predict $\tilde y = 0.3$ and $\sigma_\mathrm{n} = 1$. 
You know that $y = 0.4$. 
Complete the following function `gaussian_loglik(...)`, then compute the (log)likelihood for this sample and show its position on the Gaussian density with a plot.

In [ ]:
def gaussian_loglik(y, y_tilde, sn2):
    # @@ COMPLETE @@ #
    return


def plot_gaussian(mu, var, plot_log=False, **kwargs):
    """A simple util to plot a gaussian pdf"""
    x = np.linspace(mu - 5 * np.sqrt(var), mu + 5 * np.sqrt(var), 200)
    y = lognormal(x, mu, var) if plot_log else np.exp(lognormal(x, mu, var))
    ax = kwargs.pop("ax", plt.gca())
    ax.plot(x, y, **kwargs)
    return ax


y_obs =   # @@ COMPLETE @@ #
y_tilde =   # @@ COMPLETE @@ #
sn2 =   # @@ COMPLETE @@ #
sample_ll =  # @@ COMPLETE @@ #

fig, ax = plt.subplots(figsize=[5, 3])
plot_gaussian(y_tilde, sn2, ax=ax, label=r"$\mathcal{N}_y(\widetilde{y}, \sigma_n^2)$")
ax.vlines(y_obs, 0, np.exp(sample_ll), color="k")
ax.vlines(
    y_tilde, 0, np.exp(gaussian_loglik(y_tilde, y_tilde, sn2)), ls="--", color="k"
)
ax.plot(y_obs, np.exp(sample_ll), "ok", label=r"Likelihood")
ax.text(y_tilde + 0.1, 0.02, r"$\widetilde{y}$")
ax.text(y_obs - 0.2, 0.02, r"$y$")
ax.set_ylim(0)
ax.legend()
plt.show()

## 3. Bayesian Linear regression

In this section, you'll implement Bayesian linear regression.
Let's start by creating the **design matrix** $\mathbf{X}$.

$$
\mathbf{X} = \left[ {\begin{array}{ccccc}
   1 & x_1^1 & \dots & x_1^K\\
   1 & x_2^1 & \dots & x_2^K\\
   \vdots &    \vdots & &   \vdots \\
   1 & x_N^1 & \dots & x_N^K\\
  \end{array} } \right]
$$

**Exercise:**
Complete the following function `build_features(...)` to build $\mathbf{X}$.
This can be done in many ways. One of them is using a double list comprehension (one index for the row and one for the column), while another one is using the numpy `column_stack()` function (recommended). In any case, inspect $\mathbf{X}$ to make sure it looks OK (show the first entries). To fit higher order polynomials, we need to add extra columns to $\mathbf{X}$, therefore build it with $K$ as a parameter.

In [ ]:
def build_features(x, K):
    # @@ COMPLETE @@ #
    return

In [ ]:
build_features(x, 2)

From the lecture notes, let's define the prior on the parameters $\mathbf{w}$ as:

\begin{equation}
\mathbf{w} \sim \mathcal{N}(\mathbf{0}, \mathbf{S}) 
\end{equation}

For sake of simplicity, assume the covariance matrix $\mathbf{S}$ to be diagonal $\mathbf{S} = \sigma_\mathrm{w}^2\mathbf{I}$.
Remember that the likelihood is defined as $p(\mathbf{y}|\mathbf{w}, \mathbf{X}, \sigma_\mathrm{n}) = \mathcal{N}(\mathbf{y}|\mathbf{X}\mathbf{w}, \sigma_\mathrm{n}^2\mathbf{I})$.
In this case, the posterior is analitic and follows this form:

\begin{equation}
p(\mathbf{w}|\mathbf{X}, \mathbf{y}, \sigma_\mathrm{n}) = \mathcal{N}\left(\frac{1}{\sigma^2_\mathrm{n}}\mathbf{\Sigma}\mathbf{X}^\top\mathbf{y}, \mathbf{\Sigma} \right)
\end{equation}
where $\mathbf{\Sigma}^{-1} = \left(\frac{1}{\sigma^2_\mathrm{n}}\mathbf{X}^\top\mathbf{X} + \mathbf{S}^{-1}  \right)$.

**Question:** What is the dimensionality of $\mathbf{\Sigma}$? How much does it cost to compute that inverse? Do you know which algorithm you should use to have numerically stable results? Remember that computing $\mathbf{A}^{-1}\mathbf{z}$ means in practice solving a linear system.

**Exercise:** Complete the following function to compute the posterior.

In [ ]:
def compute_posterior(X, y, sw2, sn2):
    # @@ COMPLETE @@ #
    return w_posterior_mean, w_posterior_cov

**Execise:** Compute the posterior over the parameters for the given regression dataset. For the moment, place $\sigma_\mathrm{w}^2=1$ and start with polynomial of order 1. Finally, print the posterior mean and covariance. Comment on the results.

In [ ]:
sw2 = 1
K = 1

X =   # @@ COMPLETE @@ #

w_posterior_mean, w_posterior_cov = # @@ COMPLETE @@ #

In [ ]:
print(w_posterior_mean)

In [ ]:
print(w_posterior_cov)

In [ ]:
x_ = np.linspace(-3, 3, 50)
y_ = np.linspace(-3, 3, 50)
X_, Y_ = np.meshgrid(x_, y_)
pos = np.empty(X_.shape + (2,))
pos[:, :, 0] = X_
pos[:, :, 1] = Y_


fig = plt.figure(figsize=plt.figaspect(0.5))
ax = fig.add_subplot(1, 2, 1, projection="3d")

rv = scipy.stats.multivariate_normal(np.zeros(2), sw2 * np.eye(2))
plot_config = dict(
    cmap="viridis", linewidth=0, antialiased=False, ccount=500, rcount=500
)
ax.plot_surface(X_, Y_, rv.pdf(pos), **plot_config)
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_zlabel("")
ax.set_title("Prior")

ax = fig.add_subplot(1, 2, 2, projection="3d")
rv = scipy.stats.multivariate_normal(w_posterior_mean, w_posterior_cov)
ax.plot_surface(X_, Y_, rv.pdf(pos), **plot_config)
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_zlabel("")

ax.set_title("Posterior")
plt.show()

## 4. Predictions
Now it's time to make predictions. 
All our motivation for being Bayesian was to be able to average predictions at $\mathbf{x}_\mathrm{new}$, for all possible $\mathbf{w}$.
This is possible by computing the following expectation:


\begin{equation}
\mathbf{E}_{p(\mathbf{w}|\mathbf{X}, \mathbf{y}, \sigma_\mathrm{n})}\mathcal{N}(\mathbf{w}^\top\mathbf{x}_\mathrm{new}, \sigma^2_\mathrm{n}) = \int \mathcal{N}(\mathbf{w}^\top\mathbf{x}_\mathrm{new}, \sigma^2_\mathrm{n}) p(\mathbf{w}|\mathbf{X}, \mathbf{y}, \sigma_\mathrm{n}) \mathrm{d}\mathbf{w}
\end{equation}


**Question:** Prove that 
$\mathbf{E}_{p(\mathbf{w}|\mathbf{X}, \mathbf{y}, \sigma_\mathrm{n})}\mathcal{N}(\mathbf{w}^\top\mathbf{x}_\mathrm{new}, \sigma^2_\mathrm{n}) = 
\mathcal{N}(\mathbf{x}_\mathrm{new}^\top\mathbf{\mu}, \sigma^2_\mathrm{n} + \mathbf{x}_\mathrm{new}^\top\mathbf{\Sigma}\mathbf{x}_\mathrm{new})$, where $\mathbf{\mu}$ and $\mathbf{\Sigma}$ are the posterior mean and covariance.

**Exercise:** 
Write a function to compute the predictive distribution. 
Usually we want to do this for multiple points, hence the for-loop.

In [ ]:
def compute_predictive(Xt, w_mean, w_cov, sn2):
    def _compute_predictive_single_point(xt_i, w_mean, w_cov, sn2):
        yt_i_mean = # @@ COMPLETE @@ #
        yt_i_var = # @@ COMPLETE @@ #
        return yt_i_mean, yt_i_var

    yt_mean, yt_var = np.zeros(len(Xt)), np.zeros(len(Xt))
    for i, xt_i in enumerate(Xt):  # Loop on all the points
        yt_mean[i], yt_var[i] = _compute_predictive_single_point(
            xt_i, w_mean, w_cov, sn2
        )

    return yt_mean, yt_var

**Exercise:** Compute and plot the predictive distribution for 100 points between -4 and +4.

In [ ]:
xt = np.linspace(-4, 4, 250)
Xt = build_features(xt, K)
y_mean, y_var =   # @@ COMPLETE @@ #


fig, ax = plt.subplots(figsize=[5, 3])
ax.scatter(x, y, edgecolor="black", linewidth=1, facecolor="xkcd:orange", zorder=10)
ax.plot(xt, y_mean, color="tab:blue", lw=3)

lb = y_mean - 2 * np.sqrt(y_var)
ub = y_mean + 2 * np.sqrt(y_var)
ax.fill_between(xt, lb, ub, color=".80", lw=0)
ax.set_title(f"Predictive distribution with order {K}")
plt.show()

**Exercise:** You can also sample from the posterior and compute the function values. A simple way to do so is to sample from the posterior over the weights and evaluate the function at any inputs.
Complete and use the utility function below: sample 30 times from the posterior and plot the corresponding functions. 


In [ ]:
def sample(Xt, w_mean, w_cov, sn2, N):
    def _sample_single(Xt, w_mean, w_cov, sn2):
        # @@ COMPLETE @@ #
        return y_sample

    samples = np.zeros((N, len(Xt)))
    for i in range(N):
        samples[i] = _sample_single(Xt, w_mean, w_cov, sn2)
    return samples


samples = sample(Xt, w_posterior_mean, w_posterior_cov, sn2, N=30)
fig, ax = plt.subplots(figsize=[5, 3])
ax.scatter(x, y, edgecolor="black", linewidth=1, facecolor="xkcd:orange", zorder=10)
ax.plot(xt, samples.T, "tab:blue", alpha=0.3)
plt.show()

**Exercise**: Try now with different polynomial order. Let's say 2, 5, 10, 15. Compute the design matrix, the posterior on $\mathbf{w}$ and the predictive posterior. What do you observe?

In [ ]:
def bayesian_linear_regression(x, y, xt, K, sn2, sw2):
    X = build_features(x, K) 
    Xt = build_features(xt, K)  
    w_posterior_mean, w_posterior_cov =  # @@ COMPLETE @@ #
    y_mean, y_var =  # @@ COMPLETE @@ #
    samples =   # @@ COMPLETE @@ #
    return y_mean, y_var, samples


poly_orders = [2, 5, 10, 12]

fig, axs = plt.subplots(2, 2, figsize=[10, 6], sharex=True, sharey=True)
axs = axs.reshape(-1)
for ax, K in zip(axs, poly_orders):
    y_mean, y_var, samples = bayesian_linear_regression(x, y, xt, K, sn2, sw2)

    lb = y_mean - 2 * np.sqrt(y_var)
    ub = y_mean + 2 * np.sqrt(y_var)

    ax.fill_between(xt, lb, ub, color=".80", lw=0)
    ax.plot(xt, samples.T, "tab:blue", alpha=0.2)
    ax.plot(xt, y_mean, color="tab:blue", lw=3)
    ax.scatter(x, y, edgecolor="black", linewidth=1, facecolor="xkcd:orange", zorder=10)
    ax.set_ylim(-10, 20)
    ax.set_title("Order %d" % K)
plt.show()

**Exercise**:
Play with the next cell and try to answer the following questions:
1. Set n=0; what is the plot showing?
2. Try to increase the number of points "observed"; what is happening to the posterior?
3. Try to increase the polynomial order; how are the functions behaving?

In [ ]:
import ipywidgets as widgets
from ipywidgets import fixed, interact, interact_manual, interactive


def animate(n, K):
    y_mean, y_var, samples = bayesian_linear_regression(x[:n], y[:n], xt, K, sn2, sw2)
    fig, ax = plt.subplots(figsize=[5, 3])
    ax.scatter(
        x[:n], y[:n], edgecolor="black", linewidth=1, facecolor="xkcd:orange", zorder=10
    )
    ax.plot(xt, samples.T, "tab:blue", alpha=0.3)
    ax.set_ylim(-10, 20)
    plt.show()


interaction = interact(
    animate,
    n=widgets.IntSlider(min=0, max=len(x), step=1, value=0, continuous_update=False),
    K=widgets.IntSlider(min=0, max=10, step=1, value=1, continuous_update=False),
)

## 5. Evaluate you model: the marginal likelihood

There are several ways in which you can compute the goodness of your model. The first is the likelihood itself.

**Question:** Compute the loglikelihood for model with order from 0 to 7 and plot it. Comment on the results.

In [ ]:
def evaluate_likelihood(x, y, K, sn2, sw2):
    y_mean, y_var, _ = bayesian_linear_regression(x, y, x, K, sn2, sw2)
    # @@ COMPLETE @@ #
    return


Ks = np.arange(1, 8)
mll = [evaluate_likelihood(x, y, K, sn2, sw2) for K in Ks]

fig, ax = plt.subplots(figsize=[5, 3])
ax.plot(Ks, mll, "-ok")
ax.margins(0.05)
ax.set_title("Log-Likelihood")
ax.set_xlabel("Model (e.g. polyn. order)")

plt.show()

**Question:** Try to answer: How likely is $\mathbf{y}$ given $\mathbf{X}$ *and* the model (‘first/second/... order polynomial’)? Is it the same likelihood as before?

So far, we’ve ignored $p(\mathbf{y}|\mathbf{X}, \sigma^2_\mathrm{n})$, the normalizing constant in Bayes rule. Being a normalization constant, it has to be equal to: 

\begin{equation}
p(\mathbf{y}|\mathbf{X}, \sigma^2_\mathrm{n}) = \int p(\mathbf{y}|\mathbf{X}, \mathbf{w}, \sigma^2_\mathrm{n})
p(\mathbf{w})\mathrm{d}\mathbf{w}
\end{equation}

We’re averaging over all values of $\mathbf{w}$ from the prior to get a value for how good the model is.

**Question:** Suppose the prior is $\mathcal{N}(\mu_0, \mathbf{\Sigma}_0)$ and the likelihood $\mathcal{N}(\mathbf{X}\mathbf{w}, \sigma^2_\mathrm{n} \mathbf{I})$. Derive the marginal likelihood (hint: don't solve the integral -- check the rules for Gaussian conditioning and marginalization) (big hint: check the lecture notes).

**Exercise:** Write a function to compute the marginal likelihood. Remember: this is a *likelihood* not a density. You should return a number not a density. For simplicity, assume $\mu_0 = 0$ and $\Sigma_0 = \sigma^2_\mathrm{w}\mathbf{I}$. Use `scipy.stats.multivariate_normal` for computing the logpdf.

In [ ]:
from scipy.stats import multivariate_normal


def marginal_likelihood(X, y, sw2, sn2):
    return  # @@ COMPLETE @@ #

**Exercise:** Do the sample plot as before, but now plot the marginal likelihood. You should see a clear pattern here; comment the result.

In [ ]:
def evaluate_marginal_likelihood(x, y, K, sn2, sw2):
    X = build_features(x, K)
    return  # @@ COMPLETE @@ #


poly_orders = range(1, 8)
mll = [evaluate_marginal_likelihood(x, y, K, sn2, sw2) for K in poly_orders]

fig, ax = plt.subplots(figsize=[5, 3])
ax.plot(poly_orders, mll, "-ok")
ax.margins(0.05)

ax.set_title("Log-Marginal Likelihood")
ax.set_xlabel("Model (e.g. polyn. order)")
plt.show()